<a href="https://colab.research.google.com/github/EdlaHasini/GenerativeAI-B39/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "Housing.csv"
df = pd.read_csv(file_path)

# Identify categorical and numerical columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = [col for col in df.columns if col not in categorical_columns + ['price']]

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Handle skewness in target variable
df['price'] = np.log1p(df['price'])

# Separate features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Normalize numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the improved ANN model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32,
          validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

# Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Training MAE: {train_mae}, Testing MAE: {test_mae}')

# Save the model
model.save("/mnt/data/housing_price_model_optimized.h5")
print("Optimized model saved as 'housing_price_model_optimized.h5'.")

# Load and use the saved model
loaded_model = tf.keras.models.load_model("/mnt/data/housing_price_model_optimized.h5")
predictions = np.expm1(loaded_model.predict(X_test[:5]))

# Display sample predictions
print("Sample Predictions:")
print(predictions)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 223.3699 - mae: 14.9242 - val_loss: 179.3544 - val_mae: 13.3718
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 155.9199 - mae: 12.4071 - val_loss: 77.9006 - val_mae: 8.7007
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 51.3808 - mae: 6.5758 - val_loss: 9.4394 - val_mae: 2.3658
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 16.2930 - mae: 3.1418 - val_loss: 7.6191 - val_mae: 2.1900
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 12.7447 - mae: 2.8158 - val_loss: 8.6603 - val_mae: 2.3653
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 13.4318 - mae: 2.9608 - val_loss: 6.9892 - val_mae: 2.0926
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 11.4574 - mae: 2.7336 - val_loss: 6.4489 - val_mae: 2.0143
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 11.6341 - mae: 2.7357 - val_loss: 5.7810 - val_mae: 1.9159
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0

Training MAE: 0.2480006068944931, Testing MAE: 0.3287722170352936
Optimized model saved as 'housing_price_model_optimized.h5'.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Sample Predictions:
[[1804296. ]
 [6942124. ]
 [3110648.8]
 [4117404.8]
 [4045059. ]]
